***
# 分析の方針
***
説明変数は一定限られており、以下の項目の前処理を具体的に検討する。<BR>

【特徴量の初期仮説】<BR>
- goal     : 調達目標金額　→金額が小さい方が成功しやすい
- deadline : 調達期限　→ 調達までの期間が長い方が成功しやすい
- category : プロジェクトのカテゴリ　→おそらく成功しやすいカテゴリと成功しづらいカテゴリがあるはず
- main_category : 上記同様
- currency : 通貨　→おそらくドルが強い
- country  : 国　　→おそらくUSが強い
- name     : プロジェクト名　→　文字数とか記号の数とか、成功しやすいキーワードとかあるとかだと面白い　（優先度低）



***
# Day1からの変更点

***
- 正規化の実施    = 済　
- 交差検証の実施   = 済　
- SVMによる分析  = 実施中
- Onehotベクトルによる説明変数の追加　＝未　特徴量エンジニアリングができず。Day3までには。。<BR>

In [74]:
# ライブラリのインポート
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import log_loss, accuracy_score, confusion_matrix
from sklearn.metrics import mean_squared_error, mean_absolute_error # 回帰問題における性能評価に関する関数
from  sklearn.metrics import accuracy_score, precision_recall_fscore_support, confusion_matrix # 回帰問題における性能評価に関する関数
from sklearn.model_selection import KFold # 交差検証法に関する関数
from sklearn.metrics import mean_absolute_error # 回帰問題における性能評価に関する関数
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV

In [53]:
# ファイルのインポート
df_kick = pd.read_csv("./ks-projects-201801.csv")

In [54]:
# データ構成の把握
print(df_kick.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 378661 entries, 0 to 378660
Data columns (total 15 columns):
ID                  378661 non-null int64
name                378657 non-null object
category            378661 non-null object
main_category       378661 non-null object
currency            378661 non-null object
deadline            378661 non-null object
goal                378661 non-null float64
launched            378661 non-null object
pledged             378661 non-null float64
state               378661 non-null object
backers             378661 non-null int64
country             378661 non-null object
usd pledged         374864 non-null float64
usd_pledged_real    378661 non-null float64
usd_goal_real       378661 non-null float64
dtypes: float64(5), int64(2), object(8)
memory usage: 43.3+ MB
None


In [15]:
#データの中身の参照
df_kick.head(n=5)

,ID,name,category,main_category,currency,deadline,goal,launched,pledged,state,backers,country,usd pledged,usd_pledged_real,usd_goal_real
0,1000002330,The Songs of Adelaide & Abullah,Poetry,Publishing,GBP,2015-10-09,1000.0,2015-08-11 12:12:28,0.0,failed,0,GB,0.0,0.0,1533.95
1,1000003930,Greeting From Earth: ZGAC Arts Capsule For ET,Narrative Film,Film & Video,USD,2017-11-01,30000.0,2017-09-02 04:43:57,2421.0,failed,15,US,100.0,2421.0,30000.00
2,1000004038,Where is Hank?,Narrative Film,Film & Video,USD,2013-02-26,45000.0,2013-01-12 00:20:50,220.0,failed,3,US,220.0,220.0,45000.00
3,1000007540,ToshiCapital Rekordz Needs Help to Complete Album,Music,Music,USD,2012-04-16,5000.0,2012-03-17 03:24:11,1.0,failed,1,US,1.0,1.0,5000.00
4,1000011046,Community Film Project: The Art of Neighborhoo...,Film & Video,Film & Video,USD,2015-08-29,19500.0,2015-07-04 08:35:03,1283.0,canceled,14,US,1283.0,1283.0,19500.00


In [16]:
#データの平均等の把握
df_kick.describe()

,ID,goal,pledged,backers,usd pledged,usd_pledged_real,usd_goal_real
count,3.786610e+05,3.786610e+05,3.786610e+05,378661.000000,3.748640e+05,3.786610e+05,3.786610e+05
mean,1.074731e+09,4.908079e+04,9.682979e+03,105.617476,7.036729e+03,9.058924e+03,4.545440e+04
std,6.190862e+08,1.183391e+06,9.563601e+04,907.185035,7.863975e+04,9.097334e+04,1.152950e+06
min,5.971000e+03,1.000000e-02,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,1.000000e-02
25%,5.382635e+08,2.000000e+03,3.000000e+01,2.000000,1.698000e+01,3.100000e+01,2.000000e+03
50%,1.075276e+09,5.200000e+03,6.200000e+02,12.000000,3.947200e+02,6.243300e+02,5.500000e+03
75%,1.610149e+09,1.600000e+04,4.076000e+03,56.000000,3.034090e+03,4.050000e+03,1.550000e+04
max,2.147476e+09,1.000000e+08,2.033899e+07,219382.000000,2.033899e+07,2.033899e+07,1.663614e+08


In [55]:
#期限は重要な判断情報と考えられるため、
#期限に関する日数情報を追加する
df_kick["period"] = ((pd.to_datetime(df_kick["deadline"])) - (pd.to_datetime(df_kick["launched"])))
df_kick["period_days"] = df_kick["period"].map(lambda x: x.days)
display(df_kick.head())

,ID,name,category,main_category,currency,deadline,goal,launched,pledged,state,backers,country,usd pledged,usd_pledged_real,usd_goal_real,period,period_days
0,1000002330,The Songs of Adelaide & Abullah,Poetry,Publishing,GBP,2015-10-09,1000.0,2015-08-11 12:12:28,0.0,failed,0,GB,0.0,0.0,1533.95,58 days 11:47:32,58
1,1000003930,Greeting From Earth: ZGAC Arts Capsule For ET,Narrative Film,Film & Video,USD,2017-11-01,30000.0,2017-09-02 04:43:57,2421.0,failed,15,US,100.0,2421.0,30000.00,59 days 19:16:03,59
2,1000004038,Where is Hank?,Narrative Film,Film & Video,USD,2013-02-26,45000.0,2013-01-12 00:20:50,220.0,failed,3,US,220.0,220.0,45000.00,44 days 23:39:10,44
3,1000007540,ToshiCapital Rekordz Needs Help to Complete Album,Music,Music,USD,2012-04-16,5000.0,2012-03-17 03:24:11,1.0,failed,1,US,1.0,1.0,5000.00,29 days 20:35:49,29
4,1000011046,Community Film Project: The Art of Neighborhoo...,Film & Video,Film & Video,USD,2015-08-29,19500.0,2015-07-04 08:35:03,1283.0,canceled,14,US,1283.0,1283.0,19500.00,55 days 15:24:57,55


In [ ]:
#散布図による確認
pd.plotting.scatter_matrix(df_kick, figsize=(15,15))
plt.show()

In [ ]:
#ヒートマップでの把握
plt.figure(figsize=(12,10))
sns.heatmap(df_kick.corr(), annot=True)
plt.show()

In [ ]:
#キーとなりそうな値を確認
display(df_kick.groupby("state").mean())
display(df_kick.groupby("category").mean())
display(df_kick.groupby("main_category").mean())
display(df_kick.groupby("currency").mean())
display(df_kick.groupby("country").mean())

In [56]:
df_kick["success"] = df_kick["state"] == "successful"

In [ ]:
display(df_kick.groupby("success").mean())

In [76]:
y = df_kick["success"].values
X = df_kick[["goal","period_days"]].values

In [68]:
#一般的には正規化の方が有効のようなので、正規化のみ検証

from sklearn.preprocessing import MinMaxScaler
mmsc = MinMaxScaler()
# 説明変数のデータを正規化する
x_train_norm = mmsc.fit_transform(X)

In [71]:
#交差検証で５分割ロジスティクス回帰で実施

n_split = 5 # グループ数を設定（今回は5分割）

#cross_valid_mae = 0
split_num = 1

# テスト役を交代させながら学習と評価を繰り返す
for train_idx, test_idx in KFold(n_splits=n_split, random_state=1234).split(X, y):
    X_train, y_train = X[train_idx], y[train_idx] #学習用データ
    X_test, y_test = X[test_idx], y[test_idx]     #テスト用データ
    
    # 学習用データを使って線形回帰モデルを学習
    clf = SGDClassifier(loss='log', penalty='none', max_iter=10000, fit_intercept=True, random_state=1234)
    clf.fit(X_train, y_train)

    # テストデータに対する予測を実行
    y_pred_test = clf.predict(X_test) 
    
    
    
    # 正答率を計算
    print("Fold %s"%split_num)
    accuracy =  accuracy_score(y_test, y_pred_test)
    print('正答率（Accuracy） = {:.3f}%'.format(100 * accuracy))
    
    # Precision, Recall, F1-scoreを計算
    precision, recall, f1_score, _ = precision_recall_fscore_support(y_test, y_pred_test)

    # 成功に関するPrecision, Recall, F1-scoreを表示
    print('適合率（Precision） = {:.3f}%'.format(100 * precision[0]))
    print('再現率（Recall） = {:.3f}%'.format(100 * recall[0]))
    print('F1値（F1-score） = {:.3f}%'.format(100 * f1_score[0]))
    
    # 予測値と正解のクロス集計（混同行列）
    conf_mat = confusion_matrix(y_test, y_pred_test)
    conf_mat = pd.DataFrame(conf_mat, 
                        index=['正解 = 失敗', '正解 = 成功'], 
                        columns=['予測 = 失敗', '予測 = 成功'])
    display(conf_mat)
    
    split_num += 1


Fold 1
正答率（Accuracy） = 54.498%
適合率（Precision） = 77.401%
再現率（Recall） = 41.193%
F1値（F1-score） = 53.770%


,予測 = 失敗,予測 = 成功
正解 = 失敗,20040,28609
正解 = 成功,5851,21233


Fold 2
正答率（Accuracy） = 59.259%
適合率（Precision） = 73.894%
再現率（Recall） = 57.143%
F1値（F1-score） = 64.448%


,予測 = 失敗,予測 = 成功
正解 = 失敗,27966,20974
正解 = 成功,9880,16912


Fold 3
正答率（Accuracy） = 64.978%
適合率（Precision） = 65.464%
再現率（Recall） = 97.228%
F1値（F1-score） = 78.245%


,予測 = 失敗,予測 = 成功
正解 = 失敗,47698,1360
正解 = 成功,25163,1511


Fold 4
正答率（Accuracy） = 64.788%
適合率（Precision） = 65.154%
再現率（Recall） = 97.813%
F1値（F1-score） = 78.211%


,予測 = 失敗,予測 = 成功
正解 = 失敗,47860,1070
正解 = 成功,25597,1205


Fold 5
正答率（Accuracy） = 55.801%
適合率（Precision） = 77.056%
再現率（Recall） = 45.377%
F1値（F1-score） = 57.118%


,予測 = 失敗,予測 = 成功
正解 = 失敗,22293,26835
正解 = 成功,6638,19966


In [78]:
#SVMの検証
#SVMはは時間がかかると聞いたのでホールドアウト法で

X2 = X.reshape(-1,1) # scikit-learnに入力するために整形
test_size = 0.2        # 全データのうち、何%をテストデータにするか（今回は20%に設定）
X2_train, X2_test, y2_train, y2_test = train_test_split(X, y, test_size=test_size, random_state=1234) # ホールドアウト法を実行（テストデータはランダム選択）



In [ ]:

# SVMの実行
C = 5
clf = SVC(C=C, kernel="linear")
clf.fit(X2_train, y2_train)

# 未知のデータを識別する
clf.predict(X2_test)

score_fold = 100 * clf.score(X, y)
print('識別精度 = {:.3f}'.format(score_fold))
    


